# Enrollment: Race and Ethnicity
Load and join enrollment files, by race and ethnicity, for 2005-06 through the current year. Include state, district and school-level information in a single dataframe

In [28]:
import pandas as pd
import re
import datetime

#Source folder for the excel files, downloaded from MDE website
folder = 'C:/Users/swastvedt/Documents/MDE Data_Documents/Enrollment/Originals/'
currentYear = datetime.date.today().year
#Earliest year we have data for
a = 2005

raceEthnicity = pd.DataFrame()
files = []

###Create a list of file names
#Account for change in file name format that occurred in 2015-16
while a < currentYear:
    if a < 2016:
        files.append(folder + str(a) + '-' + str(a+1) + ' Enrollment Gender Ethnicity Rev.xlsx')
        
    else:
        files.append(folder + str(a) + '-' + str(a+1) + ' Enrollment by Ethnicity_Gender.xlsx')
        
    a = a + 1
    
def upcase_first_letter(s):
    return s[0].upper() + s[1:]

def remove_underscore(s):
    return re.sub(r'_', '', s)

###Capitalize column names and create "DistrictID" column to use as an index
def clean_table(df):
    df.columns = [str(upcase_first_letter(x)) for x in df.columns]
    df.columns = [str(remove_underscore(x)) if (x == 'Total_Male' or x == 'Total_Female') else x for x in df.columns]
    df['DistrictID'] = df['DistrictNumber'] + '-' + df['DistrictType'] + '-' + df['SchoolNumber']
    df = df.set_index(['DataYear', 'DistrictID', 'Grade'])
    
    return df

###Load files from source folder. Create a list of dataframes for each year that includes state, district and school-level information
def load_data(f):
    df_st = pd.read_excel(f, sheetname='State', dtype = {0:str, 1: str, 3: str, 4: str, 6: str, 8: str, 9: str, 25: str, 26: str, 27: str})
    df_dist = pd.read_excel(f, sheetname='District', dtype = {0:str, 1: str, 3: str, 4: str, 6: str, 8: str, 9: str, 25: str, 26: str, 27: str})
    df_sch = pd.read_excel(f, sheetname='School', dtype = {0:str, 1: str, 3: str, 4: str, 6: str, 8: str, 9: str, 25: str, 26: str, 27: str})
    
    df_list = [clean_table(d) for d in [df_st, df_dist, df_sch]]

    return df_list

In [32]:
###Load data for each file name in the files list
frames = [ load_data(f) for f in files ]

In [33]:
###Join state, district and school-level data for each year in the frames list
listCollapse = [ l[0].append([l[1], l[2]]) for l in frames ]

In [34]:
###Join dataframes for all years into one. Drop blank rows.
raceEthnicity = raceEthnicity.append(listCollapse)

In [35]:
raceEthnicity.drop('end of worksheet', level='DataYear', inplace=True)

In [36]:
###Save raceEthnicity table to local folder
raceEthnicity.to_csv('tempdata/raceEthnicity.csv')

In [37]:
###Update shared file on OneDrive and return new link to file.
import onedrivesdk
from onedrivesdk.helpers import GetAuthCodeServer

redirect_uri = 'http://localhost:8080/'

#Credentials for "Python data upload" OneDrive app
client_secret = ***
client = onedrivesdk.get_default_client(client_id=***,
                                            scopes=['wl.signin',
                                                    'wl.offline_access',
                                                    'onedrive.readwrite'])
auth_url = client.auth_provider.get_auth_url(redirect_uri)

# Block thread until we have the code
code = GetAuthCodeServer.get_auth_code(auth_url, redirect_uri)
# Finally, authenticate!
client.auth_provider.authenticate(code, redirect_uri, client_secret)

127.0.0.1 - - [24/Sep/2017 21:02:31] "GET /?code=M5a10fc05-b3ba-98de-2af3-ee4a634ca442 HTTP/1.1" 200 -


In [38]:
returned_item = client.item(id='root').children['raceEthnicity.csv'].upload('tempdata/raceEthnicity.csv')

In [39]:
#Input the client information from above and the "raceEthnicity.csv" filename to generate a new sharing link
def get_sharing_link(c, fileName):
    permission = c.item(id='root').children[fileName].create_link("view").post()
    return ("\n{}\n".format(permission.link.web_url))

get_sharing_link(client, 'raceEthnicity.csv')

'\nhttps://1drv.ms/u/s!AvY_lznp4oOFa_iN1oRnuoQ9c8k\n'